In [2]:
print("Test")

Test


In [70]:
import pandas as pd
df = pd.read_csv("venv/ner_dataset.csv", encoding="latin1")
df = df.fillna(method="ffill")
df.head(7)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo


In [71]:
# Replace tag by integers
modes_tag = df.Tag.unique()
n_tags = len(modes_tag)
tag2idx={t:i for i,t in enumerate(modes_tag)}
df['Tag'] = df.Tag.replace(tag2idx)
df.head(7)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,0
1,Sentence: 1,of,IN,0
2,Sentence: 1,demonstrators,NNS,0
3,Sentence: 1,have,VBP,0
4,Sentence: 1,marched,VBN,0
5,Sentence: 1,through,IN,0
6,Sentence: 1,London,NNP,1


In [72]:
print(df.shape)
df = df.drop(['POS'], axis=1)
df = df.groupby('Sentence #').agg(list)
df = df.reset_index(drop=True)
df.head(7)

(1048575, 4)


,Word,Tag
0,"[Thousands, of, demonstrators, have, marched, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,"[Iranian, officials, say, they, expect, to, ge...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Helicopter, gunships, Saturday, pounded, mili...","[0, 0, 7, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 5, ..."
3,"[They, left, after, a, tense, hour-long, stand...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[U.N., relief, coordinator, Jan, Egeland, said...","[1, 0, 0, 3, 10, 0, 7, 0, 1, 0, 2, 0, 2, 0, 0,..."
5,"[Mr., Egeland, said, the, latest, figures, sho...","[3, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
6,"[He, said, last, week, 's, tsunami, and, the, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [73]:
len_max = max(df.Word.apply(len))
print("Maximum size of a sentence :", len_max)


Maximum size of a sentence : 104


In [74]:
from sklearn.model_selection import train_test_split
X_text_train, X_text_test, y_train, y_test = train_test_split(df.Word, df.Tag, test_size=0.2, random_state=1234)


In [75]:
y_test

8872     [0, 0, 0, 0, 0, 7, 0, 1, 0, 3, 0, 0, 0, 0, 0, ...
36510    [0, 0, 0, 7, 0, 0, 0, 1, 4, 0, 1, 0, 3, 10, 0,...
15093    [0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, ...
26251    [5, 0, 5, 6, 0, 0, 0, 5, 0, 0, 0, 3, 5, 0, 0, ...
6517     [0, 2, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, ...
                               ...                        
26232              [0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
9251     [0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
3302     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
24304    [3, 10, 0, 0, 1, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0,...
46087    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
Name: Tag, Length: 9592, dtype: object

In [76]:
import tensorflow as tf
# Définition du tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
# Mettre à jour le dictionnaire du tokenizer
tokenizer.fit_on_texts(X_text_train)
# Définition des dictionnaires
word2idx = tokenizer.word_index
idx2word = tokenizer.index_word
vocab_size = tokenizer.num_words

In [77]:
X_train = tokenizer.texts_to_sequences(X_text_train)
X_test = tokenizer.texts_to_sequences(X_text_test)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=len_max, padding='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=len_max, padding='post')
y_train = tf.keras.preprocessing.sequence.pad_sequences(y_train, maxlen=len_max, padding='post', value=n_tags)
y_test = tf.keras.preprocessing.sequence.pad_sequences(y_test, maxlen=len_max, padding='post', value=n_tags)


In [83]:
print(X_train[:5,:], y_train[:5,:])
print('Shape of sentence X :', X_train.shape)
print('Shape of tags y :', y_train.shape)



[[   4   80    9 1125    3 2107    4  135 2929    9  126    7 2074 8168
  1024    8    7 3801 8168 1024    8 1742  191    4    1  574    2    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [1633  619   30 1536  476    6 1093  226    5  792    4   37 3468 1353
    14   15 6790    1   76    9 2345   21 1996    2    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0

In [89]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, RNN, GRUCell, Conv2D, Embedding, Bidirectional, Dropout

model = Sequential()
model.add(Embedding(vocab_size, vocab_size))
model.add(Bidirectional(RNN(GRUCell(256, recurrent_initializer='glorot_uniform'),
                                return_sequences=True)))
model.add(Dropout(0.3))
# la sortie a n_tags+1 units, c'est à dire le nombre de tags possibles plus la classe utililisée pour égalisée
# la taille des data (qui ne sera pas prise en compte dans l'entraînement grâce à la fonction de coût customisée)
model.add(Dense(n_tags+1, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 10000)       100000000 
_________________________________________________________________
bidirectional (Bidirectional (None, None, 512)         15756288  
_________________________________________________________________
dropout (Dropout)            (None, None, 512)         0         
_________________________________________________________________
dense (Dense)                (None, None, 18)          9234      
Total params: 115,765,522
Trainable params: 115,765,522
Non-trainable params: 0
_________________________________________________________________


In [90]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

def loss_function(real, pred):
    # Mask => vaudra 0 lorsque y vaut n_tags, c'est à dire la valeur avec laquelle on a compléter le tableau y_train
    mask = tf.math.logical_not(tf.math.equal(real, n_tags))
    # Pour respecter le type de y
    mask = tf.cast(mask, dtype=pred.dtype)
    # fonction de perte
    loss_ = loss_object(real, pred)
    # Apply mask on loss function
    loss_ *= mask
    # on renvoit la moyenne calculée en dehors de mask = 0, valeur que l'on ne veut pas prendre en compte ds le calcul
    return tf.reduce_mean(loss_)

In [91]:
model.compile(loss=loss_function, optimizer='adam')


In [ ]:
history = model.fit(X_train, y_train, batch_size=16, epochs=1)

In [ ]:
# prediction sur les 10 premières phrases
y_prob = model.predict(X_test[:10])
y_pred = y_prob.argmax(axis=2)


In [ ]:
def target2text(X_int, y_pred):
    X = []
    for i in range(len(y_pred)):
        text=''
        for j in np.flip(np.arange(len(y_pred[0]))):
            if y_pred[i, j]==1 :
                text = ' ' + text
            text = idx2word[X_int[i, j]] + text
        X.append(text)
    return X

print('Prediction :')
target2text(X_test[:len(y_pred)], y_pred)

In [ ]:
print("OK")